# Tensorflow Probability Utilities

This notebook is a bit of a mess after the refactor.
Its code has all been moved to `indl.model.tfp` and `indl.model.tfp.dsae`

Many of the tests have been moved to the unit tests.

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfkl
from tensorflow.keras import backend as K
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers
tfb = tfp.bijectors
scale_shift = np.log(np.exp(1) - 1).astype(np.float32)
from indl.model.tfp.devae import *
from indl.model.tfp import *


In [118]:
# An example of how this would work in a variational autoencoder.
N_TIMES = 10
N_SENSORS = 8
N_SAMPLES = 2
N_HIDDEN = 5
KL_WEIGHT = 0.05

t_vec = tf.range(N_TIMES, dtype=tf.float32) / N_TIMES
sig_vec = 1 + tf.exp(-10*(t_vec - 0.5))


def make_model(prior):
    input_ = tfkl.Input(shape=(LATENT_SIZE,))

    # Encoder
    make_latent_dist_fn, latent_params = make_mvn_dist_fn(
        input_, LATENT_SIZE, offdiag=True, loc_name="latent_loc")
    q_latent = tfpl.DistributionLambda(
        name="q_latent",
        make_distribution_fn=make_latent_dist_fn,
        convert_to_tensor_fn=lambda s: s.sample(N_SAMPLES),
        activity_regularizer=tfpl.KLDivergenceRegularizer(prior,
                                                          use_exact_kl=True,
                                                          weight=KL_WEIGHT)
    )(latent_params)

    # Decoder
    y_ = q_latent[..., tf.newaxis, :] / sig_vec[:, tf.newaxis]
    # broadcast-add zeros to restore timesteps
    #y_ = q_latent[..., tf.newaxis, :] + tf.zeros([N_TIMES, 1])
    #y_ = tf.reshape(y_, [-1, N_TIMES, LATENT_SIZE])
    #y_ = tfkl.LSTM(N_HIDDEN, return_sequences=True)(y_)
    #y_ = tf.reshape(y_, [N_SAMPLES, -1, N_TIMES, N_HIDDEN])
    make_out_dist_fn, out_dist_params = make_mvn_dist_fn(y_, N_SENSORS, loc_name="out_loc")
    p_out = tfpl.DistributionLambda(
        make_distribution_fn=make_out_dist_fn, name="p_out")(out_dist_params)
    # no prior on the output.

    # Model
    model = tf.keras.Model(inputs=input_, outputs=[q_latent, p_out])
    return model

In [119]:
# Create a fake dataset to train the model.
LATENT_SIZE = 4
BATCH_SIZE = 6
# The latents are sampled from a distribution with known parameters.
true_dist = tfd.MultivariateNormalDiag(
    loc=[-1., 1., 5, -5],  # must have length == LATENT_SIZE
    scale_diag=[0.5, 0.5, 0.9, 0.2]
)
# They parameterize sigmoid end points,
from indl.misc.sigfuncs import sigmoid
from functools import partial
t_vec = (np.arange(N_TIMES, dtype=np.float32) / N_TIMES)[None, :]
f_sig = partial(sigmoid, t_vec, B=10, x_offset=0.5)
# which are then mixed with a known mixing matrix
mix_mat = np.array([
    [-0.3, -.28, -0.38, -0.45, -0.02, -0.12, -0.05, -0.48],
    [0.27, 0.29, -0.34, 0.2, 0.41, 0.08, 0.11, 0.13],
    [-0.14, 0.26, -0.28, -0.14, 0.1, -0.2, 0.4, 0.11],
    [-0.05, -0.12, 0.28, 0.49, -0.12, 0.1, 0.17, 0.22]
], dtype=np.float32).T
#mix_mat = tf.convert_to_tensor(mix_mat)


def gen_ds(n_iters=1e2, latent_size=LATENT_SIZE):
    iter_ix = 0
    while iter_ix < n_iters:
        _input = tf.ones((latent_size,), dtype=tf.float32)
        latent = true_dist.sample().numpy()
        _y = np.reshape(latent, [latent_size, 1])
        _y = f_sig(K=_y)
        _y = mix_mat @ _y
        _y = _y.T
        yield _input, _y
        iter_ix += 1

        
ds = tf.data.Dataset.from_generator(gen_ds, args=[1e2], output_types=(tf.float32, tf.float32),
                                    output_shapes=((LATENT_SIZE,), (N_TIMES, N_SENSORS)))
ds = ds.map(lambda x, y: (x, (tf.zeros(0, dtype=tf.float32), y))).batch(BATCH_SIZE)

In [120]:
# Train the model.
# Try playing around with the 2nd loss_weights (below) and KL_WEIGHT (above).
N_EPOCHS = 100

K.clear_session()
prior = make_mvn_prior(LATENT_SIZE, trainable_mean=True, trainable_var=True, offdiag=False)
model_ = make_model(prior)

model_.compile(optimizer='adam',
               loss=[lambda _, model_latent: tfd.kl_divergence(model_latent, prior),
                     lambda y_true, model_out: -model_out.log_prob(y_true)],
               loss_weights=[0.0, 1.0])
hist = model_.fit(ds, epochs=N_EPOCHS, verbose=2)

Epoch 1/100
17/17 - 0s - loss: 91.9389 - q_latent_loss: 4.6523 - p_out_loss: 90.5613
Epoch 2/100
17/17 - 0s - loss: 11531.2471 - q_latent_loss: 4.3536 - p_out_loss: 11529.9590
Epoch 3/100
17/17 - 0s - loss: 743.5315 - q_latent_loss: 4.1376 - p_out_loss: 742.3065
Epoch 4/100
17/17 - 0s - loss: 137048.2969 - q_latent_loss: 3.9767 - p_out_loss: 137047.1250
Epoch 5/100
17/17 - 0s - loss: 113.2025 - q_latent_loss: 3.8272 - p_out_loss: 112.0695
Epoch 6/100
17/17 - 0s - loss: 74.9116 - q_latent_loss: 3.7352 - p_out_loss: 73.8058
Epoch 7/100
17/17 - 0s - loss: 79.9207 - q_latent_loss: 3.6573 - p_out_loss: 78.8380
Epoch 8/100
17/17 - 0s - loss: 372.0323 - q_latent_loss: 3.5850 - p_out_loss: 370.9711
Epoch 9/100
17/17 - 0s - loss: 60.3690 - q_latent_loss: 3.5167 - p_out_loss: 59.3279
Epoch 10/100
17/17 - 0s - loss: 7418.3252 - q_latent_loss: 3.4512 - p_out_loss: 7417.3027
Epoch 11/100
17/17 - 0s - loss: 3513.5659 - q_latent_loss: 3.3824 - p_out_loss: 3512.5649
Epoch 12/100
17/17 - 0s - loss: 53.

Epoch 96/100
17/17 - 0s - loss: 156.0881 - q_latent_loss: 2.1253 - p_out_loss: 155.4590
Epoch 97/100
17/17 - 0s - loss: 8.8568 - q_latent_loss: 2.1240 - p_out_loss: 8.2280
Epoch 98/100
17/17 - 0s - loss: 10.9563 - q_latent_loss: 2.1228 - p_out_loss: 10.3279
Epoch 99/100
17/17 - 0s - loss: 10.1586 - q_latent_loss: 2.1217 - p_out_loss: 9.5306
Epoch 100/100
17/17 - 0s - loss: 14.0194 - q_latent_loss: 2.1206 - p_out_loss: 13.3917


In [121]:
lat_wts = model_.get_layer("latent_loc").weights
lat_locs = np.ones((1, LATENT_SIZE)) @ lat_wts[0].numpy() + lat_wts[1].numpy()
mix_wts = model_.get_layer("out_loc").weights
model_out = lat_locs @ mix_wts[0].numpy() + mix_wts[1].numpy()
true_out = mix_mat @ true_dist.mean().numpy()

print(f"Model est lat: {lat_locs}")
print(f"Model est out: {model_out}")
print(f"prior mean: {prior.mean().numpy()}")
print(f"true lat: {true_dist.mean().numpy()}")
print(f"true out: {true_out.T}")


Model est lat: [[ 0.51195845 -1.02728739  0.67845761 -0.11073773]]
Model est out: [[ 0.10048061  1.36010552  0.30864524 -0.09840383  1.17217551 -0.67099368
   0.95406085  0.03414997]]
prior mean: [ 0.5117957  -0.8991166   0.66152537 -0.11197621]
true lat: [-1.  1.  5. -5.]
true out: [ 0.12000006  2.4699998  -2.76       -2.5         1.53       -1.3
  1.31        0.05999994]


In [56]:
# test LearnableMultivariateNormalDiag
prior_factory = LearnableMultivariateNormalDiag(LATENT_SIZE)
learnable_prior = prior_factory()
sample = learnable_prior.sample((100, 64))
print(sample.shape)
print(learnable_prior.trainable_variables)

(100, 64, 4)
(<tf.Variable 'learnable_multivariate_normal_diag_2/mean:0' shape=(4,) dtype=float32, numpy=array([ 0.16748714, -0.1799583 ,  0.0387747 ,  0.11378615], dtype=float32)>, <tf.Variable 'learnable_multivariate_normal_diag_2/transformed_scale:0' shape=(4,) dtype=float32, numpy=array([-0.11407143,  0.06062925,  0.02439827, -0.01735771], dtype=float32)>)


In [58]:
K.clear_session()
model_ = make_model(learnable_prior)

model_.compile(optimizer='adam',
               loss=[lambda _, model_latent: tfd.kl_divergence(model_latent, learnable_prior),
                     lambda y_true, model_out: -model_out.log_prob(y_true)],
               loss_weights=[0.0, 1.0])
print(learnable_prior.trainable_variables)
print([_.name for _ in model_.trainable_variables])

hist = model_.fit(ds, epochs=N_EPOCHS, verbose=2)

lat_wts = model_.get_layer("latent_loc").weights
lat_locs = np.ones((1, LATENT_SIZE)) @ lat_wts[0].numpy() + lat_wts[1].numpy()
mix_wts = model_.get_layer("out_loc").weights
model_out = lat_locs @ mix_wts[0].numpy() + mix_wts[1].numpy()
true_out = mix_mat @ true_dist.mean().numpy()

print(f"Model est lat: {lat_locs}")
print(f"Model est out: {model_out}")
print(f"prior mean: {learnable_prior.mean().numpy()}")
print(f"true lat: {true_dist.mean().numpy()}")
print(f"true out: {true_out.T}")

(<tf.Variable 'learnable_multivariate_normal_diag_2/mean:0' shape=(4,) dtype=float32, numpy=array([ 0.16748714, -0.1799583 ,  0.0387747 ,  0.11378615], dtype=float32)>, <tf.Variable 'learnable_multivariate_normal_diag_2/transformed_scale:0' shape=(4,) dtype=float32, numpy=array([-0.11407143,  0.06062925,  0.02439827, -0.01735771], dtype=float32)>)
['dense/kernel:0', 'dense/bias:0', 'latent_loc/kernel:0', 'latent_loc/bias:0', 'dense_1/kernel:0', 'dense_1/bias:0', 'out_loc/kernel:0', 'out_loc/bias:0', 'learnable_multivariate_normal_diag_2/mean:0', 'learnable_multivariate_normal_diag_2/transformed_scale:0']
Epoch 1/100
17/17 - 0s - loss: 266.6295 - q_latent_loss: 6.5640 - p_out_loss: 264.6859
Epoch 2/100
17/17 - 0s - loss: 2032.0966 - q_latent_loss: 6.2853 - p_out_loss: 2030.2358
Epoch 3/100
17/17 - 0s - loss: 83.5799 - q_latent_loss: 6.0718 - p_out_loss: 81.7824
Epoch 4/100
17/17 - 0s - loss: 82.7522 - q_latent_loss: 5.8942 - p_out_loss: 81.0072
Epoch 5/100
17/17 - 0s - loss: 59.2224 - q

17/17 - 0s - loss: 21.3484 - q_latent_loss: 2.0557 - p_out_loss: 20.7399
Epoch 90/100
17/17 - 0s - loss: 11.3400 - q_latent_loss: 2.0439 - p_out_loss: 10.7350
Epoch 91/100
17/17 - 0s - loss: 14.2551 - q_latent_loss: 2.0345 - p_out_loss: 13.6529
Epoch 92/100
17/17 - 0s - loss: 14.2384 - q_latent_loss: 2.0268 - p_out_loss: 13.6385
Epoch 93/100
17/17 - 0s - loss: 16.3489 - q_latent_loss: 2.0194 - p_out_loss: 15.7511
Epoch 94/100
17/17 - 0s - loss: 14.2265 - q_latent_loss: 2.0118 - p_out_loss: 13.6310
Epoch 95/100
17/17 - 0s - loss: 11.5992 - q_latent_loss: 2.0041 - p_out_loss: 11.0060
Epoch 96/100
17/17 - 0s - loss: 11.7333 - q_latent_loss: 1.9971 - p_out_loss: 11.1421
Epoch 97/100
17/17 - 0s - loss: 12.1329 - q_latent_loss: 1.9904 - p_out_loss: 11.5438
Epoch 98/100
17/17 - 0s - loss: 13.7211 - q_latent_loss: 1.9832 - p_out_loss: 13.1341
Epoch 99/100
17/17 - 0s - loss: 37.2112 - q_latent_loss: 1.9745 - p_out_loss: 36.6267
Epoch 100/100
17/17 - 0s - loss: 9.3972 - q_latent_loss: 1.9630 - p

# Latent Dynamic Factor

In [ ]:
# Return 3 outputs, the first 2 are null
#ds_dyn = ds.map(lambda x, y: (x, (y[0], y[0], y[1])))
ds_dyn = ds.map(lambda x, y: (x, y[1]))
KL_WEIGHT = 0.001
LATENT_SIZE_DYNAMIC = 1  # Integer dimensionality of each dynamic, time-variant latent variable `z_t`.

In [ ]:
K.clear_session()
tmp = LearnableMultivariateNormalDiagCell(3, 4)
#tmp.build((None, 10, 5))
#tmp.summary()

In [ ]:
# test DynamicEncoder and LearnableMultivariateNormalDiagCell
K.clear_session()
dynamic_encoder = DynamicEncoder(N_HIDDEN, N_TIMES, LATENT_SIZE_DYNAMIC)
sample, dynamic_prior = dynamic_encoder.sample_dynamic_prior(
    N_TIMES, samples=N_SAMPLES, batches=1)
print(sample.shape)
print("mean:", np.squeeze(dynamic_prior.mean()))
print("stddev:", np.squeeze(dynamic_prior.stddev()))
print([_.name for _ in dynamic_encoder.trainable_variables])


(2, 1, 10, 1)
mean: [[ 0.          0.42388976  0.45631832  0.365768    0.20130846  0.37873474
   0.31262326  0.26073667  0.15399611  0.14049806]
 [ 0.          0.08804662  0.0361465  -0.03267653 -0.08733355  0.19941618
   0.30335566  0.3730844   0.2744042   0.17948757]]
stddev: [[1.00001    0.929902   0.9554563  0.99371654 1.0142238  0.97018814
  1.0006421  1.0033575  1.0105829  1.0065393 ]
 [1.00001    0.9952911  1.0008274  1.0000954  0.99874425 0.98230976
  0.9771384  0.9683764  1.000174   1.0049998 ]]
['learnable_multivariate_normal_diag_cell/mvndiagcell_lstm/kernel:0', 'learnable_multivariate_normal_diag_cell/mvndiagcell_lstm/recurrent_kernel:0', 'learnable_multivariate_normal_diag_cell/mvndiagcell_lstm/bias:0', 'learnable_multivariate_normal_diag_cell/mvndiagcell_loc/kernel:0', 'learnable_multivariate_normal_diag_cell/mvndiagcell_loc/bias:0', 'learnable_multivariate_normal_diag_cell/mvndiagcell_scale/kernel:0', 'learnable_multivariate_normal_diag_cell/mvndiagcell_scale/bias:0']


In [ ]:
K.clear_session()
f_model = FactorizedAutoEncoder(N_HIDDEN, N_TIMES, LATENT_SIZE, LATENT_SIZE_DYNAMIC, N_SENSORS)
# Most of the trainable variables don't present themselves until the model pieces are called.
print([_.name for _ in f_model.static_encoder.trainable_variables])
print([_.name for _ in f_model.dynamic_encoder.trainable_variables])
print([_.name for _ in f_model.decoder.trainable_variables])

[]
['learnable_multivariate_normal_diag/mean:0', 'learnable_multivariate_normal_diag/untransformed_stddev:0']
[]
[]


In [ ]:
N_EPOCHS = 200
if False:
    f_model.compile(optimizer='adam',
                    loss=lambda y_true, model_out: -model_out.log_prob(y_true))
    hist = f_model.fit(ds_dyn, epochs=N_EPOCHS, verbose=2)
else:
    @tf.function
    def grad(model, inputs, preds):
        with tf.GradientTape() as tape:
            q_f = model.static_encoder(inputs)
            q_z = model.dynamic_encoder(inputs)
            p_full = model.decoder([tf.convert_to_tensor(q_f),
                                    tf.convert_to_tensor(q_z)])

            # Reconstruction log-likelihood: p(output|input)
            recon_post_log_prob = p_full.log_prob(preds)
            recon_post_log_prob = tf.reduce_sum(recon_post_log_prob,
                                                axis=-1)  # Sum over time axis
            recon_post_log_prob = tf.reduce_mean(recon_post_log_prob)

            # KL Divergence - analytical
            # Static
            static_prior = model.static_encoder.static_prior_factory()
            stat_kl = tfd.kl_divergence(q_f, static_prior)
            stat_kl = KL_WEIGHT * stat_kl
            stat_kl = tf.reduce_mean(stat_kl)
            
            # Dynamic
            _, dynamic_prior = model.dynamic_encoder.sample_dynamic_prior(
                N_TIMES, samples=1, batches=1
            )
            dyn_kl = tfd.kl_divergence(q_z, dynamic_prior)
            dyn_kl = tf.reduce_sum(dyn_kl, axis=-1)
            dyn_kl = tf.squeeze(dyn_kl)
            dyn_kl = KL_WEIGHT * dyn_kl
            dyn_kl = tf.reduce_mean(dyn_kl)
            
            loss = -recon_post_log_prob + stat_kl + dyn_kl

        grads = tape.gradient(loss, model.trainable_variables)
        return loss, grads, (-recon_post_log_prob, stat_kl, dyn_kl)

    optim = tf.keras.optimizers.Adam(learning_rate=1e-3)
    for epoch_ix in range(N_EPOCHS):
        for step_ix, batch in enumerate(ds_dyn):
            inputs, preds = batch
            loss, grads, loss_comps = grad(f_model, inputs, preds)
            optim.apply_gradients(zip(grads, f_model.trainable_variables))
            if (step_ix % 200) == 0:
                print('.')
        print(f"Epoch {epoch_ix}/{N_EPOCHS}:\tloss={loss:.3f}; "
              f"Losses: {[_.numpy() for _ in loss_comps]}")

.
Epoch 0/200:	loss=3777.438; Losses: [3777.3972, 0.0028260916, 0.03819199]
.
Epoch 1/200:	loss=908.620; Losses: [908.5834, 0.002681077, 0.034407526]
.
Epoch 2/200:	loss=682.733; Losses: [682.7001, 0.0025911012, 0.030505255]
.
Epoch 3/200:	loss=317.985; Losses: [317.9555, 0.0024803109, 0.027142774]
.
Epoch 4/200:	loss=737.753; Losses: [737.7305, 0.0024128703, 0.019867169]
.
Epoch 5/200:	loss=293.983; Losses: [293.9585, 0.0023704215, 0.022514882]
.
Epoch 6/200:	loss=412.075; Losses: [412.0592, 0.002335041, 0.013460781]
.
Epoch 7/200:	loss=306.618; Losses: [306.60327, 0.002284743, 0.012861049]
.
Epoch 8/200:	loss=215.916; Losses: [215.9029, 0.0022479186, 0.0111077]
.
Epoch 9/200:	loss=223.136; Losses: [223.12366, 0.0022122823, 0.010420884]
.
Epoch 10/200:	loss=300.156; Losses: [300.14383, 0.0021807426, 0.009667382]
.
Epoch 11/200:	loss=179.766; Losses: [179.75607, 0.0021481065, 0.007676568]
.
Epoch 12/200:	loss=215.981; Losses: [215.97124, 0.0021149626, 0.008112778]
.
Epoch 13/200:	loss=

Epoch 106/200:	loss=109.324; Losses: [109.32308, 0.00047426036, 0.00046340926]
.
Epoch 107/200:	loss=113.328; Losses: [113.327286, 0.0004729222, 0.00046490182]
.
Epoch 108/200:	loss=117.106; Losses: [117.10452, 0.000471713, 0.00062898267]
.
Epoch 109/200:	loss=122.371; Losses: [122.37039, 0.0004705812, 0.00051386596]
.
Epoch 110/200:	loss=119.422; Losses: [119.42122, 0.0004696009, 0.0005573735]
.
Epoch 111/200:	loss=131.784; Losses: [131.78348, 0.000468354, 0.00041559048]
.
Epoch 112/200:	loss=124.476; Losses: [124.475006, 0.00046699354, 0.00041292777]
.
Epoch 113/200:	loss=104.487; Losses: [104.486534, 0.00046530017, 0.00042556314]
.
Epoch 114/200:	loss=119.418; Losses: [119.41684, 0.0004641684, 0.00039730535]
.
Epoch 115/200:	loss=117.776; Losses: [117.77547, 0.00046339296, 0.00056288636]
.
Epoch 116/200:	loss=112.189; Losses: [112.18817, 0.0004628609, 0.00045002214]
.
Epoch 117/200:	loss=116.317; Losses: [116.31613, 0.0004616853, 0.00036934114]
.
Epoch 118/200:	loss=159.105; Losses:

In [ ]:
_, dyn_prior = f_model.dynamic_encoder.sample_dynamic_prior(10)
np.squeeze(dyn_prior.mean().numpy())

array([-1.6796231, -2.5568666, -2.4644861, -2.4013069, -2.3772488,
       -2.3707619, -2.377105 , -2.332031 , -2.3327737, -2.360692 ],
      dtype=float32)

In [ ]:
K.clear_session()
dynamic_prior = RNNMultivariateNormalDiag(VariationalLSTMCell(N_HIDDEN,
                                                              output_dim=LATENT_SIZE_DYNAMIC),
                                          n_timesteps=N_TIMES, output_dim=LATENT_SIZE_DYNAMIC)
sample = dynamic_prior.sample((N_SAMPLES, BATCH_SIZE))
print(sample.shape)
print(dynamic_prior.mean())

(2, 6, 10, 1)
tf.Tensor(
[[ 0.        ]
 [ 0.04328619]
 [ 0.08498121]
 [-0.17377347]
 [-0.09743058]
 [-0.30255282]
 [-0.22110605]
 [-0.36379734]
 [-0.30933833]
 [-0.13590682]], shape=(10, 1), dtype=float32)
